In [1]:
#import OpenOPC as OP
import time
import threading as th
#import wx
import datetime
import serial
import pywintypes
pywintypes.datetime=pywintypes.TimeType
import socket

### Definicion de variables globales

In [2]:
#lista de comandos:
#cerrar: finaliza el programa
close="5"
#iniciar: inicia la medicion
start= "1"
#detener: detiene la medicion
stop="2"
#envia una nueva configuracion
config="0"
#Chequea el estado de la aplicacion
status="4"
#Diccionario para mensajes luego de los comandos
comdic={config:"Cargando configuraciones", start:"Mediciones iniciadas", stop:"Mediciones detenidas", close:"Cerrando programa", status: "Estado de sistema:"}
comando=config
comprev=comando
variables=[]
device=serial.Serial()
HOST = "192.168.99.10"  # The server's hostname or IP address
PORT = 9999  # The port used by the server

# Configuraciones estandar
repeticiones =10 #-> Número de repeticiones


### Funciones


In [18]:
## Funcion que carga configuraciones ->Listo en TEST
def configurar(archivo="configuracion/config.txt"):
    "Lee la configuración de un archivo de text que se da como argumento. Las frecuencias van entre $...$, separando los valores con comas"
    try:
        with open(archivo) as config:
            texto="".join(config.readlines())
            ubic1=texto.find("$")+1
            ubic2=texto.find("$",ubic1)
            if ubic1>=0 and ubic2>0:
                frecuencias=texto[ubic1+1:ubic2-1].split(",")
                for i, f in enumerate(frecuencias): 
                    frecuencias[i]=int(f)
            else: 
                frecuencias=[]
                print("No se encuentra configuración de frecuencia")
    except Exception as e:
        print(e)
    return frecuencias

In [ ]:
##Funcion que chequea estado
def check_estado():
    "Funcion que chequea estado del sistema"
    global comprev, device, comdic
    
    if device.isOpen():
        dsp="Dispositivo conectado\n"
    else:
        dsp="Dispositivo desconectado\n"
        
    cmd=comdic[comprev]+"\n"
    
    estado="\n\n -Dispositivo de medición: "+dsp+" -Comando actual: "+cmd
    
    return estado

In [ ]:
## Funcion que permite ingresar comandos desde consola ->OBSOLETA en Test
def comandos():
    "Funcion que permite ingresar comandos desde consola"
    global comando, close, comdic, comprev
    while comando!=close:
        comando1 =input("\ningrese comando:\n  0. Cargar nueva configuracion\n  1. Iniciar medicion\n  2. Detener medicion\n  4. Estado\n  5. Salir\n")
        if comando1 in comdic:
            comprev=comando
            comando=comando1
            print(comdic[comando])
        else:
            print("el comando {} no se reconoce".format(comando1))

In [22]:
def medir_LCR(parametro=0):
    "Rutina que toma las mediciones del instrumento serie"
    global comando,start, variables, device
    #Acá va la funcion que mide las variables de CAPEM y del instrumento 
    escritura=0
    #Lectura de LCR
        
    wr=bytes("121","utf-8") #-->Reemplazar por los comandos al LCR
    device.write(wr)
    time.sleep(0.5)
    resp=device.read(3)
    print(resp.decode('utf-8'))
    #print(datetime.datetime.now())
    
    return resp.decode('utf-8')

In [2]:
def leer_server(): #->Listo en TEST
    global servidor
    servidor.connect((HOST, PORT))
           
    envio="datos".encode("ascii")
    servidor.sendall(envio)
    data = servidor.recv(1024).decode("ascii")
    print(str(data))
    time.sleep(0.5)
        

def interpretar_posicion():
    res=1
    return res

### Rutina principal

In [ ]:
##### Concectar puerto serie

#if device.port==None:
    #device=serial.Serial(port="COM3",baudrate=19200,bytesize=8, parity="N", stopbits=1, timeout=10)
#elif device.isOpen()==False:
   # device.open()
##### Thread 2, Busca comandos
th2=th.Thread(target=comandos,name="th2")
th2.start()
servidor=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
## Loop principal
while comando != close:
    if comando == status:
        sts=check_estado()
        print(sts)
        comando=comprev
    if comando == start:
        leer_server()
        interpretar_posicion()
        try:
            for freq in frecuencias:
                #setear_LCR()
                time.sleep(2) #-> Tiempo de estabilización del LCR
                for rep in enumerate(repeticiones):
                    medir_LCR()                    
        except StopIteration:
            pass
    if comando == config:
        frecuencias=configurar()
        comando=stop
        

print("Programa finalizado")
if device.isOpen() == True:
    device.close()
   

    